# University Project II - Big Data

Author : Ophiase

In [3]:
ENABLE_DOWNLOAD = False
ENABLE_UNZIP = False
ENABLE_COMPUTE_ZIP_TO_PARQUET = False

### Dependencies

In [26]:
import shutil
import os
import requests
import pandas as pd
import re
import pyspark.sql.functions as F
import zipfile
from functools import reduce
import math

In [29]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import matplotlib.pyplot as plt
import pyspark
import logging

logging.getLogger("pyspark").setLevel(logging.ERROR)
# os.environ["PYSPARK_SUBMIT_ARGS"] = "--driver-memory 2g"

import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import when, mean, stddev, skewness, kurtosis, expr, date_format
import pyspark.sql.functions as pf
from pyspark.sql.functions import col, lit, udf, dayofweek, avg
from pyspark.sql.types import DoubleType


import altair as alt
import plotly
import plotly.express as px

import scipy
from scipy.stats import skew, kurtosis

import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [6]:
spark = SparkSession.builder \
    .appName("Big Data Project") \
    .config("spark.driver.memory", "3g") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/20 17:51:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Download the DATA

In [7]:
# from 2014 to 2023
trip_urls = [
    (year, f"https://s3.amazonaws.com/tripdata/{year}-citibike-tripdata.zip")
    for year in range(2014, 2023 + 1)
]

if not os.path.exists(os.path.join('data')) :
    os.makedirs('data')

zip_files = []

for year, url in trip_urls:
    basename = os.path.join('data', str(year) + "_" + 'citibike_tripdata')
    zip_filename = basename + ".zip"
    csv_filename = basename + ".csv"
    zip_files.append((year, zip_filename, csv_filename))

    if not ENABLE_DOWNLOAD : continue
    print(f'Check {basename} ...')

    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024
    progress = 0

    if not os.path.exists(zip_filename) and not os.path.exists(csv_filename) :
        with open(zip_filename, 'wb') as f:
            for data in response.iter_content(block_size):
                if data:
                    f.write(data)
                    progress += len(data)
                    print(f'\rDownloaded {progress}/{total_size} bytes', end='')

        print(f'\nDownload complete: {zip_filename}')

print("Finished")


Finished


In [8]:
if ENABLE_UNZIP :
    for (year, zip_filename, csv_filename) in zip_files:
        # if year < 2018: continue # WARNING : DISABLE THIS LINE

        if not zipfile.is_zipfile(zip_filename):
            print("Corrupted zip file.")
            break

        if os.path.exists("tmp"):
            shutil.rmtree("tmp")

        print("Unzip : ", zip_filename)
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall("tmp")
        print("Process ..")

        # find the folder in tmp
        items = os.listdir("tmp")
        for folder in items :
            if not os.path.isdir(os.path.join("tmp", folder)) or \
                folder.startswith("__") :
                continue
            
            # find all the folder in this folder
            sub_folders = os.listdir(os.path.join("tmp", folder))
            for sub_folder in sub_folders :
                if not os.path.isdir(os.path.join("tmp", folder, sub_folder)) or \
                    sub_folder.startswith(".") : 
                    continue
                
                sub_item = os.listdir(os.path.join("tmp", folder, sub_folder))
                for leaf in sub_item :
                    # move the csv inside to data
                    from_path = os.path.join("tmp", folder, sub_folder, leaf)
                    dest_path = os.path.join("data", leaf)
                    if os.path.exists(dest_path) : 
                        os.remove(dest_path)

                    shutil.move(from_path, "data")

    if os.path.exists("tmp"):
        shutil.rmtree("tmp")

To optimize disk usage, we could have unziped one file at a time and convert its content instantaneously to `.parquet`.

## Convert the Data

### Raw Analysis

In [9]:
csv_reader = spark.read.option("header", "true") \
            .option("inferSchema", "true").csv

In [10]:
def find_all_csv():
    all_csv = []
    for item in os.listdir("data"):
        if not item.endswith(".csv") :
            continue
        all_csv.append(item)
    return sorted(all_csv)

all_csv = find_all_csv()

if False: # check column_names.txt
    for item in all_csv:    
        df = csv_reader(os.path.join("data", item))
        print(f"item {item} : {df.columns}")

By looking at the previous code output *(cached in `column_names.txt`)*, \
we notice the following columns between 2014-01 $\to$ 2021-01 (included) :
- `['tripduration', 'starttime', 'stoptime', 'start station id', 'start station name', `\
`'start station latitude', 'start station longitude', 'end station id', 'end station name', `\
`'end station latitude', 'end station longitude', 'bikeid', 'usertype', 'birth year', 'gender']`
    - The naming convention is not exactly the same between : `201610-citibike-tripdata_1.csv` $\to$ `201703-citibike-tripdata.csv_1.csv` : \
    `['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID',` \
    `'Start Station Name', 'Start Station Latitude', 'Start Station Longitude',` \
    `'End Station ID', 'End Station Name', 'End Station Latitude',` \
    `'End Station Longitude', 'Bike ID', 'User Type', 'Birth Year', 'Gender']`

The columns change between 2021-02 $\to$ 2023-12 (included) :
- `['ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_name', `\
`'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', `\
`'start_lng', 'end_lat', 'end_lng', 'member_casual']`

We decide the following matching (**O.** as before 2021-02, **N.** as after 2021-02):

- `O.'tripduration'` as function : `stoptime` - `startime`
    - Renamed as `trip_duration`

- `N.'started_at'` $\leftarrow$ `O.'starttime'`
- `N.'ended_at'` $\leftarrow$ `O.'stoptime'` 
- `N.'start_station_id'` $\leftarrow$ `O.'start station id'`
    - type : string
- `N.'start_station_name'` $\leftarrow$ `O.'start station name'`
- `N.'start_lat'` $\leftarrow$ `O.'start station latitude'`
- `N.'start_lng'` $\leftarrow$ `O.'start station longitude'`
- `N.'end_station_id'` $\leftarrow$ `O.'end station id'`
    - type : string
- `N.'end_station_name'` $\leftarrow$ `O.'end station name'`
- `N.'end_lat'` $\leftarrow$ `O.'end station latitude'`
- `N.'end_lng'` $\leftarrow$ `O.'end station longitude'`
`
- `N.'ride_id'` $\leftarrow$ `O.'bikeid'` (format is not the same)
    - Both type of ID can registered as string

- `N.'member_casual'` $\leftarrow$ `O.'usertype'` (format is not the same)
    - Mapping : `O.Subscriber`, `O.Customer` $\to$ `N.member`, `N.casual`

- `O.'birth year` : (None) for elements of N
    - Renamed as `birth_year`
- `O.'gender'` : (None) for elements of N
- `N.'rideable_type'` : (None) for elements of O

- We will also add a binary column `old_format` to indicate if the data comes from `O` or `N` as defined above.

In [11]:
col_mapping_1 = {
    'tripduration': 'trip_duration',
    'usertype': 'member_casual',
    'birth year': 'birth_year',

    'starttime': 'started_at',
    'stoptime': 'ended_at',
    'start station id': 'start_station_id',
    'start station name': 'start_station_name',
    'start station latitude': 'start_lat',
    'start station longitude': 'start_lng',
    'end station id': 'end_station_id',
    'end station name': 'end_station_name',
    'end station latitude': 'end_lat',
    'end station longitude': 'end_lng',
    'bikeid': 'ride_id',
}

col_mapping_2 = {
    'Trip Duration': 'tripduration',
    'Start Time': 'starttime',
    'Stop Time': 'stoptime',
    
    'Start Station ID': 'start station id',
    'Start Station Name': 'start station name',
    'Start Station Latitude': 'start station latitude',
    'Start Station Longitude': 'start station longitude',

    'End Station ID': 'end station id',
    'End Station Name' : 'end station name',
    'End Station Latitude' : 'end station latitude',
    'End Station Longitude' : 'end station longitude',
    
    'Bike ID' : 'bikeid',
    'User Type' : 'usertype',
    'Birth Year' : 'birth year',
    'Gender' : 'gender'
}


In [12]:
def check_unique_values(df, column):
    return df.select(column).dropDuplicates().rdd.map(lambda row: row[0]).collect()

In [13]:
def fast_check():
    df_o = spark.read.csv(os.path.join("data", all_csv[0]), header=True, inferSchema=True)
    df_o = df_o.select(
        [col(old_col).alias(col_mapping_1.get(old_col, old_col)) for old_col in df_o.columns]
        )

    df_n = spark.read.csv(os.path.join("data", all_csv[-1]), header=True, inferSchema=True)
    
    print(check_unique_values(df_o, "member_casual"))
    print(check_unique_values(df_o, "gender"))
    print(check_unique_values(df_n, "rideable_type"))

    df_o.printSchema()
    df_n.printSchema()
    
    # df.show()

fast_check()

['Subscriber', 'Customer']
[1, 2, 0]
['electric_bike', 'classic_bike']
root
 |-- trip_duration: integer (nullable = true)
 |-- started_at: timestamp (nullable = true)
 |-- ended_at: timestamp (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_lat: double (nullable = true)
 |-- start_lng: double (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_lat: double (nullable = true)
 |-- end_lng: double (nullable = true)
 |-- ride_id: integer (nullable = true)
 |-- member_casual: string (nullable = true)
 |-- birth_year: string (nullable = true)
 |-- gender: integer (nullable = true)

root
 |-- ride_id: string (nullable = true)
 |-- rideable_type: string (nullable = true)
 |-- started_at: timestamp (nullable = true)
 |-- ended_at: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: string (nullable = 

### File format Selection : Parquet

<img src="resources/file-formats.png" alt="Drawing" style="width: 400px;"/>

Our [course](https://stephane-v-boucheron.fr/slides/tbd/slides06_file-formats.html#/title-slide) on Big Data file formats.

Parquet suits our needs for the project:

- Suitable for laptop execution
- 37GB dataset size manageable with Parquet's compression
- Supports splitability for processing subsets of data on minimal configuration
- Compatible with Apache Spark

### CSV $\to$ Parquet

In [14]:
if not os.path.exists(os.path.join('computed')) :
    os.makedirs('computed')

In [15]:
factorial_columns = ["member_casual", "gender", "rideable_type"]

In [16]:
# Lower Case O. part

# add missing columns
def csv_o_process(df):
    df = df.select(
        [col(old_col).alias(col_mapping_1.get(old_col, old_col)) for old_col in df.columns])

    df = df.withColumn("birth_year", 
                       when(col("birth_year") == r"\N", None)
                       .otherwise(col("birth_year"))
                       .cast("integer")
                       )

    df = df.withColumn("start_station_id", col("start_station_id").cast("string"))\
            .withColumn("end_station_id", col("end_station_id").cast("string")) \
            .withColumn("ended_at", col("ended_at").cast("timestamp")) \
            .withColumn("started_at", col("started_at").cast("timestamp")) \
            .withColumn("ride_id", col("ride_id").cast("string"))

    df = df.withColumn("member_casual",
                        when(col("member_casual") == "Subscriber", lit("member")) \
                        .otherwise(lit("casual")))
    
    df = df.withColumn("rideable_type", lit(None).cast('string'))
    df = df.withColumn("old_format", lit(True))
    
    df = df.select(*sorted(df.columns))

    # df.write.mode("append").parquet(parquet_file)
    return df

def csv_to_parquet_part_1(csv_file) :
    df = spark.read.csv(csv_file, header=True, inferSchema=True)
    return csv_o_process(df)

In [17]:
# Upper case O. part

def csv_to_parquet_part_2(csv_file) :
    df = spark.read.csv(csv_file, header=True, inferSchema=True)
    
    df = df.select(
        [col(old_col).alias(col_mapping_2.get(old_col, old_col)) for old_col in df.columns])
    
    return csv_o_process(df)

In [18]:
# N. Part

def csv_to_parquet_part_3(csv_file) :
    df = spark.read.csv(csv_file, header=True, inferSchema=True)

    df = df.withColumn("trip_duration", lit(None).cast("integer")) # TODO
    df = df.withColumn("old_format", lit(False))
    df = df.withColumn("birth_year", lit(None).cast("integer"))
    df = df.withColumn("gender", lit(0)) # TODO verify the default value
    df = df.withColumn("old_format", lit(False))

    df = df.withColumn("start_station_id", col("start_station_id").cast("string"))\
            .withColumn("end_station_id", col("end_station_id").cast("string")) \
            .withColumn("ended_at", col("ended_at").cast("timestamp")) \
            .withColumn("started_at", col("started_at").cast("timestamp")) \
            .withColumn("ride_id", col("ride_id").cast("string"))

    df = df.select(*sorted(df.columns))

    # df.write.mode("append").parquet(parquet_file)
    return df


#### Manual Partitioning

In [19]:
# Scheduler

schedule = [
    ("201401-citibike-tripdata_1.csv", 1),
    ("201610-citibike-tripdata_1.csv", 2),
    ("201704-citibike-tripdata.csv_1.csv", 1),
    ("202102-citibike-tripdata_1.csv", 3)
]

# The file size are always below 180M
# We can safely load simultaneously 20 files from memory to .parquet
CSV_PER_PARQUET = 10

def reduce_df_array(df_array):
    # if not df_array:
    #     raise ValueError("Empty array.")

    # schema = df_array[0].schema
    # for df in df_array:
    #     if df.schema != schema:
    #         raise ValueError("Incompatibles schema.")

    return reduce(lambda df1, df2: df1.union(df2), df_array)

def process_csv_files(csv_files, schedule, parquet_dir="computed", max_size_gb=1, start_with=None):
    schedule_pointer = 0
    parquet_index = 0

    df_buffer = []

    # ignore = start_with is not None
    for index, csv_file in enumerate(csv_files):
        # if ignore:
        #     if csv_file != start_with : continue
        #     ignore = False
        #     parquet_index = index // CSV_PER_PARQUET

        csv_path = os.path.join("data", csv_file)
        
        print(f"[] {csv_file}")
        if (schedule_pointer < len(schedule) - 1) and (csv_file == schedule[schedule_pointer + 1][0]) :
            schedule_pointer += 1
            print(f"{csv_file} | {schedule_pointer} : {schedule[schedule_pointer]}")

        schedule_mode = schedule[schedule_pointer][1]
        target_function = [csv_to_parquet_part_1, csv_to_parquet_part_2, csv_to_parquet_part_3][schedule_mode - 1]

        df_buffer.append(target_function(csv_path))

        if (index + 1) % CSV_PER_PARQUET == 0 :
            parquet_path = os.path.join(
                parquet_dir, 'precompiled', f'part_{parquet_index:04d}.parquet')
            print(f"Writing ({index}/{len(csv_files)}) : {parquet_dir}")
            print(f"From {csv_files[index-CSV_PER_PARQUET+1]} to {csv_files[index]}")

            df = reduce_df_array(df_buffer)
            df_buffer = []
            parquet_index += 1

            df.write.mode("overwrite").parquet(parquet_path)

if ENABLE_COMPUTE_ZIP_TO_PARQUET:
    process_csv_files(all_csv, schedule) #, start_with="202011-citibike-tripdata_2.csv")

### Verifying Parquet is working

In [20]:
# df = spark.read.parquet(os.path.join("computed", "precompiled", "part_0023.parquet"))
df = spark.read.parquet(os.path.join("computed", "precompiled", "*.parquet"))
print(df.count())
df.show()

195252960
+----------+-----------------+------------------+--------------+--------------------+-------------------+------+-------------+----------+----------------+-------------+-----------------+------------------+----------------+--------------------+-------------------+-------------+
|birth_year|          end_lat|           end_lng|end_station_id|    end_station_name|           ended_at|gender|member_casual|old_format|         ride_id|rideable_type|        start_lat|         start_lng|start_station_id|  start_station_name|         started_at|trip_duration|
+----------+-----------------+------------------+--------------+--------------------+-------------------+------+-------------+----------+----------------+-------------+-----------------+------------------+----------------+--------------------+-------------------+-------------+
|      NULL|40.72970805644994| -73.9865979552269|       5746.02|     E 10 St & 2 Ave|2023-09-03 10:24:16|     0|       member|     false|B0A0F1DEFA4B72FC|el

## Analysis

In [40]:
end_date_pre_covid = '2020-02-29'
start_date_post_covid = '2020-03-01'

df_pre_covid = df.filter(col('ended_at') <= end_date_pre_covid)
df_2018 = df.filter((col('ended_at') <= '2019-01-01') & (col('started_at') > '2018-01-01')) 

df_post_covid = df.filter(col('started_at') >= start_date_post_covid)
df_2022 = df.filter((col('ended_at') <= '2023-01-01') & (col('started_at') > '2022-01-01')) 

In [24]:
print("Before Covid-19")
df_pre_covid.show(5)

Before Covid-19


+----------+-----------+------------+--------------+--------------------+--------------------+------+-------------+----------+-------+-------------+------------------+------------------+----------------+--------------------+--------------------+-------------+
|birth_year|    end_lat|     end_lng|end_station_id|    end_station_name|            ended_at|gender|member_casual|old_format|ride_id|rideable_type|         start_lat|         start_lng|start_station_id|  start_station_name|          started_at|trip_duration|
+----------+-----------+------------+--------------+--------------------+--------------------+------+-------------+----------+-------+-------------+------------------+------------------+----------------+--------------------+--------------------+-------------+
|      1993|40.71850211|-73.98329859|         349.0|Rivington St & Ri...|2018-12-20 14:12:...|     1|       member|      true|  34649|         NULL|       40.72580614|      -73.97422494|           339.0|  Avenue D & E 12

In [25]:
print("After Covid-19")
df_post_covid.show(5)

After Covid-19
+----------+-----------------+-----------------+--------------+--------------------+-------------------+------+-------------+----------+----------------+-------------+-----------------+------------------+----------------+--------------------+-------------------+-------------+
|birth_year|          end_lat|          end_lng|end_station_id|    end_station_name|           ended_at|gender|member_casual|old_format|         ride_id|rideable_type|        start_lat|         start_lng|start_station_id|  start_station_name|         started_at|trip_duration|
+----------+-----------------+-----------------+--------------+--------------------+-------------------+------+-------------+----------+----------------+-------------+-----------------+------------------+----------------+--------------------+-------------------+-------------+
|      NULL|40.72970805644994|-73.9865979552269|       5746.02|     E 10 St & 2 Ave|2023-09-03 10:24:16|     0|       member|     false|B0A0F1DEFA4B72FC|e

### Trip distance by day

In [28]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Rayon de la Terre en kilomètres
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c

haversine_udf = udf(haversine, DoubleType())

In [42]:
def compute_trip_distance_over_week(df, verbose=False):
    if verbose : print("compute distance")

    df = df.filter(
        col("start_lat").isNotNull() & 
        col("start_lng").isNotNull() & 
        col("end_lat").isNotNull() & 
        col("end_lng").isNotNull()
    )

    df = df.withColumn(
        "distance_km", haversine_udf(col("start_lat"), 
                                     col("start_lng"), 
                                     col("end_lat"), 
                                     col("end_lng")))
    
    if verbose : print("compute day of week")

    df = df.withColumn("day_of_week", dayofweek(col("started_at")))

    if verbose : print("group by")
    
    avg_distance_by_day_of_week = df.groupBy("day_of_week") \
        .agg(avg("distance_km").alias("avg_distance_km"))
    
    avg_distance_by_day_of_week = avg_distance_by_day_of_week.withColumn(
        "day_of_week",
        when(col("day_of_week") == 1, "Sunday")
        .when(col("day_of_week") == 2, "Monday")
        .when(col("day_of_week") == 3, "Tuesday")
        .when(col("day_of_week") == 4, "Wednesday")
        .when(col("day_of_week") == 5, "Thursday")
        .when(col("day_of_week") == 6, "Friday")
        .when(col("day_of_week") == 7, "Saturday")
    )

    return avg_distance_by_day_of_week

In [43]:
print("Trip distance over week in 2018")
compute_trip_distance_over_week(df_2018).show()

Trip distance over week in 2018


+-----------+------------------+
|day_of_week|   avg_distance_km|
+-----------+------------------+
|     Sunday|1.7461603350028398|
|     Friday|1.7766569111752966|
|    Tuesday|1.7983723666188383|
|   Thursday|  1.79577718680504|
|  Wednesday|1.8015392269836976|
|   Saturday|1.7828920533192332|
|     Monday|1.7913233909698618|
+-----------+------------------+



In [44]:
print("Trip distance over week in 2022")
compute_trip_distance_over_week(df_2022).show()

Trip distance over week in 2022


+-----------+------------------+
|day_of_week|   avg_distance_km|
+-----------+------------------+
|     Sunday| 1.875066031662407|
|     Friday|1.8119837277899957|
|    Tuesday|1.8201703663119915|
|   Thursday|1.8254055881989868|
|  Wednesday|1.8318979345853759|
|   Saturday|1.9080280784761932|
|     Monday|1.7956361630068163|
+-----------+------------------+



#### Count the number of trips for each pickup/dropoff location couple

#### Compute trip distance distribution for gender (when available)

#### Compute trip distance distribution for age ranges (15-24,25-44, 45-54, 55-64, 65+)
(when available)

#### Compute trip distance distribution for different kind of bikes (when available)

## Monitoring

## Spatial Informations